<a href="https://colab.research.google.com/github/FatmaelzahraaKhamiss/DEPI_FinalProject_SupplyChainAnalysis/blob/main/Final_Project_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retail Stores Inventory and Demand - Data Cleaning

### Introduction
Before analysis, we need to ensure our data is clean, so we imported our data into Python to verify its quality for analysis.

## Data Gathering

In [43]:
# Necessary imports
import pandas as pd
import numpy as np

In [44]:
# Reading .csv file & Show it
fd = pd.read_csv('Fact_Demand.csv')

print(fd.head())

   Store_Key  Product_Key  Date_Key  Weather_Key  Promo_Event_Key  \
0          1            1  20220101            1                1   
1          1           21  20220101            1                1   
2          1           39  20220101            1                1   
3          1            2  20220101            1                2   
4          1           22  20220101            1                2   

   Inventory_Level  Units_Sold  Units_Ordered  Price  Competitor_Pricing  \
0              195         102          252.0  72.72               85.73   
1              195         102          252.0  72.72               85.73   
2              195         102          252.0  72.72               85.73   
3              117         117          249.0  80.16               92.02   
4              117         117          249.0  80.16               92.02   

   Demand  
0     115  
1     115  
2     115  
3     229  
4     229  


## Display Information & Clean Data

In [45]:
# Print database information
print(fd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243200 entries, 0 to 243199
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Store_Key           243200 non-null  int64  
 1   Product_Key         243200 non-null  int64  
 2   Date_Key            243200 non-null  int64  
 3   Weather_Key         243200 non-null  int64  
 4   Promo_Event_Key     243200 non-null  int64  
 5   Inventory_Level     243200 non-null  int64  
 6   Units_Sold          243200 non-null  int64  
 7   Units_Ordered       93668 non-null   float64
 8   Price               243200 non-null  float64
 9   Competitor_Pricing  243200 non-null  float64
 10  Demand              243200 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 20.4 MB
None


Convert Key Columns to Categorical Data Types

In [46]:
# Define the key identifier columns
key_columns = ['Store_Key', 'Product_Key', 'Weather_Key', 'Promo_Event_Key']

In [47]:
# Convert data type for these columns
for col in key_columns:
    # Use 'Int64' first to handle potential NaN values as integers
    # then convert to 'category' for efficient storage and analysis
    fd[col] = fd[col].astype('Int64').astype('category')

In [48]:
# Verify the new data types
print("\nData Types after Conversion:")
print(fd[key_columns].dtypes)


Data Types after Conversion:
Store_Key          category
Product_Key        category
Weather_Key        category
Promo_Event_Key    category
dtype: object


Comprehensive Missing Value Check

In [49]:
# Calculate missing counts and percentages for all columns
missing_data = fd.isnull().sum()
missing_percentage = (missing_data / len(fd)) * 100

In [50]:
# Create a summary DataFrame and filter for columns with missing values
missing_summary = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_percentage
}).sort_values(by='Missing_Count', ascending=False)

In [51]:
# Display only the columns that still have missing values
print("\nComprehensive Missing Value Summary:")
print(missing_summary[missing_summary['Missing_Count'] > 0])


Comprehensive Missing Value Summary:
               Missing_Count  Missing_Percentage
Units_Ordered         149532           61.485197


Fill in the missing data from Units_Ordered column with the median


In [52]:
# Calculate the median
units_ordered_median = fd['Units_Ordered'].median()

In [53]:
# Use .fillna() to impute the missing values with the calculated median
fd['Units_Ordered'].fillna(units_ordered_median, inplace=True)

/tmp/ipython-input-394481259.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fd['Units_Ordered'].fillna(units_ordered_median, inplace=True)


In [54]:
# Print database information again
print(fd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243200 entries, 0 to 243199
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   Store_Key           243200 non-null  category
 1   Product_Key         243200 non-null  category
 2   Date_Key            243200 non-null  int64   
 3   Weather_Key         243200 non-null  category
 4   Promo_Event_Key     243200 non-null  category
 5   Inventory_Level     243200 non-null  int64   
 6   Units_Sold          243200 non-null  int64   
 7   Units_Ordered       243200 non-null  float64 
 8   Price               243200 non-null  float64 
 9   Competitor_Pricing  243200 non-null  float64 
 10  Demand              243200 non-null  int64   
dtypes: category(4), float64(3), int64(4)
memory usage: 13.9 MB
None


Make sure there are no duplicates


In [55]:
duplicate_rows = fd[fd.duplicated(keep=False)]
print(duplicate_rows)

Empty DataFrame
Columns: [Store_Key, Product_Key, Date_Key, Weather_Key, Promo_Event_Key, Inventory_Level, Units_Sold, Units_Ordered, Price, Competitor_Pricing, Demand]
Index: []


Check for Business Logic Consistency (Inventory vs. Sales)

In [56]:
# 1. Identify rows where Units_Sold > Inventory_Level
inconsistent_sales = fd[fd['Units_Sold'] > fd['Inventory_Level']]

print(f"Number of inconsistent rows (Units_Sold > Inventory_Level) = {len(inconsistent_sales)}")

Number of inconsistent rows (Units_Sold > Inventory_Level) = 0


In [57]:
# Display the inconsistent rows for manual review (Crucial for clarification)
print("\nInconsistent Rows Found (Units Sold Exceeds Inventory):")
print(inconsistent_sales)


Inconsistent Rows Found (Units Sold Exceeds Inventory):
Empty DataFrame
Columns: [Store_Key, Product_Key, Date_Key, Weather_Key, Promo_Event_Key, Inventory_Level, Units_Sold, Units_Ordered, Price, Competitor_Pricing, Demand]
Index: []


In [58]:
# Add a final statement to clarify the next action
if len(inconsistent_sales) > 0:
    print(f"\nACTION REQUIRED: There are {len(inconsistent_sales)} rows where sales exceed inventory.")
else:
    print("\nRESULT: All rows are logically consistent (Units Sold <= Inventory Level). No further action needed for this check.")


RESULT: All rows are logically consistent (Units Sold <= Inventory Level). No further action needed for this check.


## Display statistical Information & Clean Data


In [59]:
# Print statistical information for the database
print(fd.describe())

           Date_Key  Inventory_Level     Units_Sold  Units_Ordered  \
count  2.432000e+05    243200.000000  243200.000000  243200.000000   
mean   2.022624e+07       297.315362      88.091143     185.502632   
std    5.660511e+03       222.867776      43.721981     120.309219   
min    2.022010e+07         0.000000       0.000000       0.000000   
25%    2.022071e+07       135.000000      57.000000     158.000000   
50%    2.023012e+07       225.000000      83.000000     158.000000   
75%    2.023072e+07       401.000000     113.000000     158.000000   
max    2.024013e+07      2267.000000     426.000000    1616.000000   

               Price  Competitor_Pricing         Demand  
count  243200.000000       243200.000000  243200.000000  
mean       68.916885           70.683825     103.432360  
std        39.105928           40.686876      46.667984  
min       -37.790000            4.290000       4.000000  
25%        33.950000           34.680000      71.000000  
50%        65.960000 

Based on the previous information, it appears that the price column contains negative values

In [60]:
# Display negative values ​​in the dataset
negative_quantities = fd[fd['Price'] < 0]
print(negative_quantities)

print(f"\nNumber of negative values: {len(negative_quantities)}")

       Store_Key Product_Key  Date_Key Weather_Key Promo_Event_Key  \
26             1          55  20220101           1               1   
34189          5          24  20220417           2               7   
175948         5           4  20230704           2               5   

        Inventory_Level  Units_Sold  Units_Ordered  Price  Competitor_Pricing  \
26                  244          42          158.0 -37.79               44.85   
34189               124          19          158.0  -4.74                5.70   
175948              164          72           72.0  -7.43                8.78   

        Demand  
26          61  
34189       28  
175948      87  

Number of negative values: 3


In [61]:
# Convert all negative Price values to their positive absolute value
# This step ensures that all prices are non-negative, which is required for retail data.
fd['Price'] = fd['Price'].abs()

In [62]:
# Print statistical information again
print(fd.describe())

           Date_Key  Inventory_Level     Units_Sold  Units_Ordered  \
count  2.432000e+05    243200.000000  243200.000000  243200.000000   
mean   2.022624e+07       297.315362      88.091143     185.502632   
std    5.660511e+03       222.867776      43.721981     120.309219   
min    2.022010e+07         0.000000       0.000000       0.000000   
25%    2.022071e+07       135.000000      57.000000     158.000000   
50%    2.023012e+07       225.000000      83.000000     158.000000   
75%    2.023072e+07       401.000000     113.000000     158.000000   
max    2.024013e+07      2267.000000     426.000000    1616.000000   

               Price  Competitor_Pricing         Demand  
count  243200.000000       243200.000000  243200.000000  
mean       68.917296           70.683825     103.432360  
std        39.105204           40.686876      46.667984  
min         4.740000            4.290000       4.000000  
25%        33.950000           34.680000      71.000000  
50%        65.960000 

## Exporting to CSV file

In [63]:
fd.to_csv("Fact_Demand_Cleaned.csv", index=False, encoding='utf-8')

## Downloading file

In [64]:
# Import the files module from google.colab for download functionality
from google.colab import files

In [65]:
files.download('Fact_Demand_Cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>